In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [4]:
print('first 100 characters:', text[:100])

first 100 characters: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
print('number of unique characters:', len(chars))
print("".join(chars))

number of unique characters: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [6]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print('encoded version of first 100 characters:', encode("hii there"))  
print('decoded version of first 100 characters:', decode(encode("hii there")))

encoded version of first 100 characters: [46, 47, 47, 1, 58, 46, 43, 56, 43]
decoded version of first 100 characters: hii there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f37359ae2c0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('t =', t, 'context =', context, 'target =', target)

t = 0 context = tensor([18]) target = tensor(47)
t = 1 context = tensor([18, 47]) target = tensor(56)
t = 2 context = tensor([18, 47, 56]) target = tensor(57)
t = 3 context = tensor([18, 47, 56, 57]) target = tensor(58)
t = 4 context = tensor([18, 47, 56, 57, 58]) target = tensor(1)
t = 5 context = tensor([18, 47, 56, 57, 58,  1]) target = tensor(15)
t = 6 context = tensor([18, 47, 56, 57, 58,  1, 15]) target = tensor(47)
t = 7 context = tensor([18, 47, 56, 57, 58,  1, 15, 47]) target = tensor(58)


In [15]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y 

xb, yb = get_batch('train')
print(xb.shape, yb.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [16]:
print(xb.shape)

torch.Size([4, 8])


In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)
        B, T, C = logits.shape
        if targets != None:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), targets.view(-1)) 
            return logits, loss
        else:
            return logits, None

    def generate(self, idx, n):
        for _ in range(n):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=-1)
        return idx


m = BigramLanguageModel(len(chars)).to(device)
out, loss = m(xb, yb)
print(out.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long).to(device), n=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)

pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [18]:
optimizer = torch.optim.AdamW(m.parameters(), lr=0.001)

In [19]:
batch = 4
for steps in range(1000):

    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())


3.8720200061798096


In [20]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long).to(device), n=100)[0].tolist()))


W?w3cHPyZWk,f's$a-oizCjmuX
YoR&$FMVofXibIcB!!BA!$W:CdYlHxcueg ireeYERnkciK;lxWiHFlivr&v EsSV&BLqWk p


In [24]:
out.shape

torch.Size([32, 65])

xbow = torch.zeros((batch_size, block_size, 65))
out = out.view(batch_size, block_size, -1)
for b in range(batch_size):
    for t in range(block_size):
        xprev = out[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [25]:
out = out.view(batch, block_size, -1)
wei = torch.tril(torch.ones((block_size, block_size))).to(device)
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ out

In [26]:
tril = torch.tril(torch.ones((block_size, block_size))).to(device)
wei = torch.zeros((block_size, block_size)).to(device)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow = wei @ out
torch.allclose(xbow, xbow2)

True

In [32]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C).to(device)

head_size = 16
key = nn.Linear(C, head_size, bias=False).to(device)
query = nn.Linear(C, head_size, bias=False).to(device)
value = nn.Linear(C, head_size, bias=False).to(device)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones((T, T))).to(device)
# wei = torch.zeros((T, T)).to(device)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
xbow = wei @ v

xbow.shape

torch.Size([4, 8, 16])